# Librerías

In [1]:
import arcpy
import json
import random
import numpy as np, numpy.random
from numpy.random import multinomial
import pandas as pd
from random import uniform

from arcgis.features import GeoAccessor, GeoSeriesAccessor

# Funciones

In [2]:
def leer_config(config_path):
    """Lee el archivo config y lo almacena cómo diccionario.

    Args:
        config_path (str): Ruta archivo config.json

    Returns:
        configa_data (dict): Diccionario que contiene los parámetros de entrada.

    """

    with open(config_path) as json_file:
        config_data = json.load(json_file)

    return config_data

In [3]:
def crear_dict_gse():
    """
    Genera diccionario de los distintos GSE's para las zonas:
    
    Returns:
        dict_gse (dict): Diccionario de los GSE a incluir dentro del Feature Class.
        
    """
    
    # Se inicializa el diccionario
    dict_gse = {"p_bajo": 0,
                "p_medio": 0,
                "p_alto": 0}
    
    return dict_gse

In [4]:
def crear_np_gse(n_rows):
    """
    Genera un numpy array de 3 dimensiones, donde la suma total de cada fila es igual a 100.
    Cada columna representa un GSE.
    
    Args:
        n_rows (int): N° de filas del Feature Class al que se le añadirán las columnas
        
    Returns:
        np_gse (np.array): Numpy array de 3 dimensiones que representa la distribución de GSE's de cada zona.
    """
    
    np_gse = np.vstack([(np.random.dirichlet(np.ones(3),size=1)*100)[0] for i in range(n_rows)])
    
    return np_gse

In [5]:
def crear_lista_random(suma_total, n_rows):
    """
    Crea una lista de tamaño n_rows donde la suma corresponde a suma_total. 
    Los valores se distribuyen según una distribución binomial.
    
    Args:
        suma_total (int): Suma total de la lista generada.
        n_rows (int): Número de elementos de lista a generar.
        
    Returns:
        lista_random (list): Lista de tamaño n_rows y suma equivalente a suma_total
    """
    
    lista_random = multinomial(suma_total, [1/n_rows] * n_rows)
    
    return lista_random

In [6]:
def crear_dict_superficies(dict_uso_suelo, n_rows):
    """Genera diccionario que contiene la cantidad de superficie repartida a cada zona.

    Args:
        dict_uso_suelo (dict): Diccionario que contiene la cantidad de superficie a repartir para cada uso.
        n_rows (int): Número de filas de la capa de zonas.

    Returns:
        dict_superficies (dict): Diccionario que contiene la lista de superficie asiganda a cada zona para cada uso.


    """
    dict_superficies = {
        "residencial": crear_lista_random(dict_uso_suelo["residencial"], n_rows),
        "comercio": crear_lista_random(dict_uso_suelo["comercio"], n_rows),
        "educacion": crear_lista_random(dict_uso_suelo["educacion"], n_rows),
        "salud": crear_lista_random(dict_uso_suelo["salud"], n_rows),
        "oficinas": crear_lista_random(dict_uso_suelo["oficinas"], n_rows),
        "parque": crear_lista_random(dict_uso_suelo["parque"], n_rows)
    }

    return dict_superficies

In [7]:
def crear_gse_proposito(n_rows):
    """
    Genera un numpy array de 5 dimensiones, donde la suma total de cada fila es igual a 1.
    Cada columna representa un el porcentaje en que se dsitribuye un gse por proposito p.
    
    Se utiliza para la generación de viajes
    
    Args:
        n_rows (int): N° de filas del Feature Class al que se le añadirán las columnas
        
    Returns:
        lista_gse_proposito (np.array): Numpy array de 3x5 dimensiones que representa la distribución de propositos por GSE en cada zona.
        lista_gse (list): Lista de gse.
    """
    
    lista_gse = ["bajo","medio","alto"]
    lista_gse_proposito = []

    for i in lista_gse:
        np_gse_proposito = np.vstack([(np.random.dirichlet(np.ones(5),size=1))[0] for i in range(n_rows)])
        lista_gse_proposito.append(np_gse_proposito)

    return lista_gse_proposito

In [8]:
def crear_lista_atraccion(dict_betas_proposito, dict_superficies, proposito):
    """
    Crea una lista de atracción para una determinado proposito.

    Args:
        dict_betas_proposito (dict): Diccionario que contiene los betas para cada proposito.
        dict_superficies (dict): Diccionario que contiene la cantidad de superficie para cada zona para cada superficie.
        proposito (str): Propósito para el cual quiere generarse la lista de atracción.

    Returns:
        lista_atraccion (list): Lista que contiene el nivel de atraccion para cada zona para el proposito dado.
    """

    lista_atraccion = dict_betas_proposito[proposito][0] + dict_betas_proposito[proposito][1] * dict_superficies['comercio'] + dict_betas_proposito[proposito][2] * dict_superficies['educacion'] + dict_betas_proposito[proposito][3] * dict_superficies['salud'] + dict_betas_proposito[proposito][4] * dict_superficies['oficinas'] + dict_betas_proposito[proposito][5] * dict_superficies['parque']

    return lista_atraccion

In [9]:
def crear_df_zonas(dict_superficies, gse_bajo, gse_medio, gse_alto):
    """
    Genera data frame de estado inicial de zonas: usos y poblacion.
    
    Returns:
        df_viajes (df): Data frame de que contiene informacion de zonas, usos y poblacion por gse
    """
    # Se inicializa el df
    df_zonas = pd.DataFrame({"residencia": dict_superficies["residencial"], # usos de suelo
                "comercio": dict_superficies["comercio"],
                "educacion": dict_superficies["educacion"],
                "salud": dict_superficies["salud"],
                "oficinas": dict_superficies["oficinas"],
                "parque": dict_superficies["parque"],
                "p_bajo": gse_bajo, # poblacion
                "p_medio": gse_medio,
                "p_alto": gse_alto})
    
    # modificar index para comenzar desde 1
    df_zonas.index += 1
    df_zonas['id'] = df_zonas.index
    
    return df_zonas

In [10]:
def crear_df_viajes():
    """
    Genera data frame de los viajes generados.
    
    Returns:
        df_viajes (df): Data frame de los viajes por GSE y proposito p a incluir dentro del Feature Class.
        
    """
    
    # Se inicializa el df
    df_viajes = pd.DataFrame({"v_b_c": v_b_comercio, # viajes gse bajo
                "v_b_e": v_b_educacion,
                "v_b_s": v_b_salud,
                "v_b_o": v_b_oficinas,
                "v_b_p": v_b_parque,
                "v_m_c": v_m_comercio, # viajes gse medio
                "v_m_e": v_m_educacion,
                "v_m_s": v_m_salud,
                "v_m_o": v_m_oficinas,
                "v_m_p": v_m_parque,
                "v_a_c": v_a_comercio, # viajes gse alto
                "v_a_e": v_a_educacion,
                "v_a_s": v_a_salud,
                "v_a_o": v_a_oficinas,
                "v_a_p": v_a_parque})
    
    # modificar index para comenzar desde 1
    df_viajes.index += 1 
    df_viajes['id'] = df_viajes.index
    
    return df_viajes

In [11]:
def crear_df_atracciones():
    """
    Genera un dta frame de las atracciones generadas.
    
    Returns:
        df_atracciones (df): Data frame de las atraciones por GSE y propósito p a incluir dentro del Feature Class.
        
    """
    
    # Se inicializa el df
    
    df_atracciones = pd.DataFrame({"a_c": a_comercio, # atraccion comercio
                "a_e": a_educacion, # atraccion educacion
                "a_s": a_salud, # atraccion salud
                "a_o": a_oficinas, # atraccion oficinas
                "a_p": a_parque})
    
    '''respaldo
    df_atracciones = pd.DataFrame({"a_b_c": a_b_comercio, # atracción gse bajo
                "a_b_e": a_b_educacion,
                "a_b_s": a_b_salud,
                "a_b_o": a_b_oficinas,
                "a_b_p": a_b_parque,
                "a_m_c": a_m_comercio, # atracción gse medio
                "a_m_e": a_m_educacion,
                "a_m_s": a_m_salud,
                "a_m_o": a_m_oficinas,
                "a_m_p": a_m_parque,
                "a_a_c": a_a_comercio, # atracción gse alto
                "a_a_e": a_a_educacion,
                "a_a_s": a_a_salud,
                "a_a_o": a_a_oficinas,
                "a_a_p": a_a_parque})
    '''
    
    # modificar index para comenzar desde 1
    df_atracciones.index += 1
    df_atracciones['id'] = df_atracciones.index
    
    return df_atracciones

In [12]:
def merge_od_matrix(od_matrix):
    """
    Genera un merge entre matrix origen destino y los data frames de usos de suelo, poblacion viajes y atracciones.
    
    Returns:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viajes y atracciones.
    """
    # realizar merge entre od_matrix y zonas
    od_matrix = od_matrix.merge(df_zonas, left_on='OriginID', right_on='id')
    # botar id para no generar redundancia
    od_matrix = od_matrix.drop(['id'], axis=1)
    # realizar merge entre od_matrix y viajes
    od_matrix = od_matrix.merge(df_viajes, left_on='OriginID', right_on='id')
    # botar id para no generar redundancia
    od_matrix = od_matrix.drop(['id'], axis=1)
    # realizar merge entre od_matrix y atracciones
    od_matrix = od_matrix.merge(df_atracciones, left_on='OriginID', right_on='id')
    # botar id para no generar redundancia
    od_matrix = od_matrix.drop(['id'], axis=1)

    return od_matrix


In [13]:
def crear_factor(od_matrix):
    """
    Genera campos para factores de balance A por zona, gse y proposito.
    
    Inputs:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje
    
    Returns:
        list_factor_balance_a (list): lista de campos con nombres de factores A por gse y proposito
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viajes y atracciones.
    """
    # crear factores de balance A por gse y proposito p
    list_factor_balance_a =["f_a_b_c", # factores para gse bajo y comercio
                      "f_a_b_e", # factor para gse bajo y educacion
                      "f_a_b_s", # factor para gse bajo y salud
                      "f_a_b_o", # factor para gse bajo y oficinas
                      "f_a_b_p", # factor para gse bajo y parque
                      "f_a_m_c", # factor para gse medio y comercio
                      "f_a_m_e", # factor para gse medio y educacion
                      "f_a_m_s", # factor para gse medio y salud
                      "f_a_m_o", # factor para gse medio y oficinas
                      "f_a_m_p", # factor para gse medio y parque
                      "f_a_a_c", # factor para gse alto y comercio
                      "f_a_a_e", # factor para gse alto y educacion
                      "f_a_a_s", # factor para gse alto y salud
                      "f_a_a_o", # factor para gse alto y oficinas
                      "f_a_a_p"] 

    for factor_name in list_factor_balance_a:
        od_matrix[factor_name] = round(random.uniform(0, .01), 7)
        
    """
    Genera campos para factores de balance B por zona, gse y proposito.
    
    Inputs:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje
    
    Returns:
        list_factor_balance_b (list): lista de campos con nombres de factores B por gse y proposito
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viajes y atracciones.
    """

    # crear factores de balance B por gse y proposito p
    list_factor_balance_b =["f_b_b_c", # factores para gse bajo y comercio
                      "f_b_b_e", # factor para gse bajo y educacion
                      "f_b_b_s", # factor para gse bajo y salud
                      "f_b_b_o", # factor para gse bajo y oficinas
                      "f_b_b_p", # factor para gse bajo y parque
                      "f_b_m_c", # factor para gse medio y comercio
                      "f_b_m_e", # factor para gse medio y educacion
                      "f_b_m_s", # factor para gse medio y salud
                      "f_b_m_o", # factor para gse medio y oficinas
                      "f_b_m_p", # factor para gse medio y parque
                      "f_b_a_c", # factor para gse alto y comercio
                      "f_b_a_e", # factor para gse alto y educacion
                      "f_b_a_s", # factor para gse alto y salud
                      "f_b_a_o", # factor para gse alto y oficinas
                      "f_b_a_p"] 

    for factor_name in list_factor_balance_b:
        od_matrix[factor_name] = round(random.uniform(0.9,1.0), 7)
        
    return list_factor_balance_a, list_factor_balance_b, od_matrix

In [14]:
def calcular_friccion(od_matrix, dict_friccion):
    """
    Calcular la friccion para todos los gse por proposito de viaje.
    
    Inputs:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje.
    
    Returns:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje con valores de friccion calculado.
    """

    # calcular fricciones
    list_fricciones =["fr_b_c", # friccion para gse bajo y comercio
                      "fr_b_e", # friccion para gse bajo y educacion
                      "fr_b_s", # friccion para gse bajo y salud
                      "fr_b_o", # friccion para gse bajo y oficinas
                      "fr_b_p", # friccion para gse bajo y parque
                      "fr_m_c", # friccion para gse medio y comercio
                      "fr_m_e", # friccion para gse medio y educacion
                      "fr_m_s", # friccion para gse medio y salud
                      "fr_m_o", # friccion para gse medio y oficinas
                      "fr_m_p", # friccion para gse medio y parque
                      "fr_a_c", # friccion para gse alto y comercio
                      "fr_a_e", # friccion para gse alto y educacion
                      "fr_a_s", # friccion para gse alto y salud
                      "fr_a_o", # friccion para gse alto y oficinas
                      "fr_a_p"] 

    for index, friccion_name in enumerate(list_fricciones):
        beta = list(dict_friccion.values())[index][0]
        utilidad_compuesta = list(dict_friccion.values())[index][1]
        od_matrix[friccion_name] = np.exp(od_matrix['Total_Length']/1000)*beta*utilidad_compuesta
        
    return list_fricciones, od_matrix

In [25]:
def crear_distribucion(od_matrix, list_fricciones, list_factor_balance_b, list_factor_balance_a, df_viajes, df_atracciones):
    """
    Calcular distribucion de viajes para todos los gse por proposito de viaje.
    
    Inputs:
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje.
        list_fricciones (list): Lista de fricciones cacluladas.
        list_factor_bance_b (list): Lista de factores de balance b por gse y proposito de viaje.
        list_factor_bance_a (list): Lista de factores de balance a por gse y proposito de viaje.
        df_viajes (df): Data frame de viajes generados por gse y proposito.
        df_atracciones (df): Data frame de atracciones generados por superficie disponible por uso de suelo.
        
    Returns:
        list_distribucion (list): Lista con nombres de campos asociados a distribucion de viajes en df od_matrix.
        od_matrix (df): Data frame de matriz origen destino con informacion de usos, poblacion por gse, viaje con valores de distribucion calculado.
    """
    
    # Distribucion
    list_distribucion =["t_b_c", # distribucion para gse bajo y comercio
                      "t_b_e", # distribucion para gse bajo y educacion
                      "t_b_s", # distribucion para gse bajo y salud
                      "t_b_o", # distribucion para gse bajo y oficinas
                      "t_b_p", # distribucion para gse bajo y parque
                      "t_m_c", # distribucion para gse medio y comercio
                      "t_m_e", # distribucion para gse medio y educacion
                      "t_m_s", # distribucion para gse medio y salud
                      "t_m_o", # distribucion para gse medio y oficinas
                      "t_m_p", # distribucion para gse medio y parque
                      "t_a_c", # distribucion para gse alto y comercio
                      "t_a_e", # distribucion para gse alto y educacion
                      "t_a_s", # distribucion para gse alto y salud
                      "t_a_o", # distribucion para gse alto y oficinas
                      "t_a_p"] 

    # viajes*atraccion*A+B*friccion

    for index, distribucion_name in enumerate(list_distribucion):

        friccion = list_fricciones[index]
        balance_b = list_factor_balance_b[index]
        balance_a = list_factor_balance_a[index]
        viajes = list(df_viajes)[index]
        atraccion = list(np.repeat(list(df_atracciones)[:-1],3))[index]
  
        od_matrix[distribucion_name] = od_matrix[viajes]*od_matrix[atraccion]*od_matrix[balance_a]*od_matrix[balance_b]*od_matrix[friccion]

    return list_distribucion, od_matrix

In [33]:
def crear_particion_modal(od_matrix):

    # lista de particion por gse y proposito
    list_particion =["pm_b_c", # friccion para gse bajo y comercio
                      "pm_b_e", # friccion para gse bajo y educacion
                      "pm_b_s", # friccion para gse bajo y salud
                      "pm_b_o", # friccion para gse bajo y oficinas
                      "pm_b_p", # friccion para gse bajo y parque
                      "pm_m_c", # friccion para gse medio y comercio
                      "pm_m_e", # friccion para gse medio y educacion
                      "pm_m_s", # friccion para gse medio y salud
                      "pm_m_o", # friccion para gse medio y oficinas
                      "pm_m_p", # friccion para gse medio y parque
                      "pm_a_c", # friccion para gse alto y comercio
                      "pm_a_e", # friccion para gse alto y educacion
                      "pm_a_s", # friccion para gse alto y salud
                      "pm_a_o", # friccion para gse alto y oficinas
                      "pm_a_p"] 

    df_pm_gse_proposito = pd.DataFrame(np.random.randint(0,100,size=(od_matrix.shape[0], len(list_particion))), columns=list_particion)
    
    df_pm_gse_proposito[list_particion] = np.exp(lista_pm_gse_proposito[list_particion])
    
    df_pm_gse_proposito[list_particion] = lista_pm_gse_proposito[list_particion].div(lista_pm_gse_proposito[list_particion].sum(axis=0), axis=1)
    
    return df_pm_gse_proposito

# Main

In [16]:
# ENTRADAS

## Se lee el archivo config
config_data = leer_config("../config.json")

## Se almacena el diccionario de tipos de uso de suelo
dict_uso_suelo = config_data["tipos_uso"]
## Se almacena el diccionario de betas para atracción de viajes
dict_betas_proposito = config_data["betas"]
## Se almacena diccionario de beta y utilidad compuesta a propositos para la generación de atracción.
dict_friccion = config_data["friccion"]
## Se almacena el diccionario de directorios
dict_paths = config_data["paths"]

## Se lee el shape de zonas
df_zonas = pd.DataFrame.spatial.from_featureclass(dict_paths["zonas"])
## Se lee la matriz OD
od_matrix = pd.read_excel(dict_paths["od_matrix"], index_col = 0)

# INICIALIZACIÓN USO Y GSE

## Se obtiene el número de filas del shape de zonas
n_rows = len(df_zonas)

## Se crea el diccionario que contiene la superficie repartida por cada zona por tipo de uso
dict_superficies = crear_dict_superficies(dict_uso_suelo, n_rows)

## Se generan arrays de porcentajes de distribución de los distintos GSE's en las zonas
np_gse = crear_np_gse(n_rows)
    
## Crear listas de poblacion segun GSE (considerando 100 por zona, esto se podria modificar)
gse_bajo = np.intc(np_gse)[:,0]
gse_medio = np.intc(np_gse)[:,1]
gse_alto = np.intc(np_gse)[:,2]

# GENERACIÓN DE VIAJES

## Crear lista de proposito de viajes asumiendo que del total de población por GSE,un % aleatorio viaja por un propósito p
lista_gse_proposito = crear_gse_proposito(n_rows)

## Crear listas de porcentaje de gse bajo que viaja con un proposito determinado p
#v_b_comercio = np.intc(lista_gse_proposito[0][:,0] * gse_bajo)
v_b_comercio = lista_gse_proposito[0][:,0] * gse_bajo
v_b_educacion = lista_gse_proposito[0][:,1] * gse_bajo
v_b_salud = lista_gse_proposito[0][:,2] * gse_bajo
v_b_oficinas = lista_gse_proposito[0][:,3] * gse_bajo
v_b_parque = lista_gse_proposito[0][:,4] * gse_bajo

## Crear listas de porcentaje de gse medio que viaja con un proposito determinado p
v_m_comercio = lista_gse_proposito[1][:,0] * gse_medio
v_m_educacion = lista_gse_proposito[1][:,1] * gse_medio
v_m_salud = lista_gse_proposito[1][:,2] * gse_medio
v_m_oficinas = lista_gse_proposito[1][:,3] * gse_medio
v_m_parque = lista_gse_proposito[1][:,4] * gse_medio

## Crear listas de porcentaje de gse alto que viaja con un proposito determinado p
v_a_comercio = lista_gse_proposito[2][:,0] * gse_alto
v_a_educacion = lista_gse_proposito[2][:,1] * gse_alto
v_a_salud = lista_gse_proposito[2][:,2] * gse_alto
v_a_oficinas = lista_gse_proposito[2][:,3] * gse_alto
v_a_parque = lista_gse_proposito[2][:,4] * gse_alto


# ATRACCIÓN DE VIAJES

## Crear listas de atracción de viajes por proposito determinado p
a_comercio = crear_lista_atraccion(dict_betas_proposito, dict_superficies, "comercio")
a_educacion = crear_lista_atraccion(dict_betas_proposito, dict_superficies, "educacion")
a_salud = crear_lista_atraccion(dict_betas_proposito, dict_superficies, "salud")
a_oficinas = crear_lista_atraccion(dict_betas_proposito, dict_superficies, "oficinas")
a_parque = crear_lista_atraccion(dict_betas_proposito, dict_superficies, "parque")



In [17]:
# MATRIZ OD

## Crear data frame de zonas
df_zonas = crear_df_zonas(dict_superficies, gse_bajo, gse_medio, gse_alto)

## Crear data frame de viajes basado en listas de viajes
df_viajes = crear_df_viajes()

## Crear data frame de atracciones basado en listas de atracciones
df_atracciones = crear_df_atracciones()

## generar merge de datos
od_matrix = merge_od_matrix(od_matrix)

In [18]:
# FACTORES DE BALANCE

# Crear lista de factores de balance y actualizar od_matrix
list_factor_balance_a, list_factor_balance_b, od_matrix = crear_factor(od_matrix)

In [19]:
# CÁLCULO DE FRICCIONES

## Calcular fricciones sobre od_matrix
list_fricciones, od_matrix = calcular_friccion(od_matrix, dict_friccion)

In [26]:
# DISTRIBUCIÓN DE VIAJES

# crea distribuciones de viajes
list_distribucion, od_matrix = crear_distribucion(od_matrix, 
                                                    list_fricciones, 
                                                    list_factor_balance_b, 
                                                    list_factor_balance_a, 
                                                    df_viajes, 
                                                    df_atracciones)



In [34]:
# PARTICIÓN MODAL

## Se genera df con partición modal por GSE y propósito
df_pm_gse_proposito = crear_particion_modal(od_matrix)

In [38]:
df_pm_gse_proposito

,pm_b_c,pm_b_e,pm_b_s,pm_b_o,pm_b_p,pm_m_c,pm_m_e,pm_m_s,pm_m_o,pm_m_p,pm_a_c,pm_a_e,pm_a_s,pm_a_o,pm_a_p
0,2.720938e-18,2.330323e-11,1.412009e-39,1.365225e-19,1.786843e-43,3.227475e-45,2.543288e-11,7.944633e-15,1.805214e-43,3.826789e-29,1.796347e-33,2.871228e-05,2.491113e-31,1.003100e-28,7.319114e-18
1,1.485582e-16,1.988476e-27,1.058783e-05,3.965823e-06,1.339849e-09,4.548639e-23,5.379388e-30,1.555000e-03,8.987632e-45,4.306484e-36,7.947281e-28,4.725501e-43,3.371355e-32,5.476743e-27,3.030068e-25
2,3.062013e-25,3.408065e-42,1.340860e-19,2.165266e-04,3.642087e-09,3.685800e-19,1.713654e-13,3.289028e-22,2.203089e-04,1.466046e-06,6.439748e-24,1.857752e-30,8.143492e-25,3.279153e-22,3.109419e-35
3,1.587563e-03,1.160199e-12,1.432907e-06,1.330385e-09,1.909505e-30,3.938821e-06,3.774575e-09,1.555000e-03,4.659822e-23,4.722633e-33,1.149381e-15,1.934618e-07,6.598740e-21,5.703348e-04,8.801998e-12
4,8.894794e-12,8.447749e-45,5.551078e-27,1.774222e-43,9.264252e-22,1.204895e-12,7.094422e-21,4.881351e-20,1.185303e-35,6.391389e-34,5.354836e-30,5.182141e-40,3.154787e-45,1.413719e-06,9.269047e-32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,1.485582e-16,5.927562e-24,8.028286e-15,2.565981e-31,7.703492e-28,2.323940e-34,3.391767e-45,3.375161e-32,7.282753e-41,6.655838e-11,2.923641e-28,3.987546e-16,1.453469e-16,3.842886e-06,7.707479e-38
39996,8.111003e-15,1.507774e-36,2.455871e-21,4.462946e-13,2.908172e-38,1.762142e-43,7.094422e-21,1.241652e-32,3.059638e-15,1.649817e-13,1.418449e-19,7.303445e-18,2.271601e-34,1.719222e-43,2.952741e-15
39997,2.907724e-05,4.494610e-33,2.878071e-05,4.579821e-23,7.024675e-31,3.275243e-12,8.755212e-25,2.159575e-14,2.610804e-31,1.693023e-23,1.493714e-39,4.795440e-10,5.714073e-04,1.270343e-42,5.193259e-40
39998,2.792194e-28,5.776293e-14,1.375975e-29,9.686922e-42,1.696807e-23,1.788222e-10,1.424953e-19,1.397296e-39,1.080856e-38,1.444664e-39,6.439748e-24,1.810343e-20,9.404287e-42,1.837243e-30,1.612142e-13
